In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("paisesTorres") \
    .getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/04 13:26:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/04 13:26:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
rdd_paises = spark.sparkContext.textFile("/home/jovyan/data/paises.csv")
rdd_paises.take(5)

['id,equipo,sigla',
 '1,30. Februar,AUT',
 '2,A North American Team,MEX',
 '3,Acipactli,MEX',
 '4,Acturus,ARG']

In [4]:
header = rdd_paises.first()
rdd_paises2 = rdd_paises.filter(lambda x: x != header).map(lambda x: x.split(","))
rdd_paises2.take(5)

[['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG']]

In [5]:
rdd_paises2 = rdd_paises.map(lambda x: x.split(","))
rdd_paises2.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [7]:
header = rdd_paises.first()
rdd_paises2 = rdd_paises.filter(lambda x: x != header).map(lambda x: x.split(","))
rdd_paises2.take(5)

[['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG']]

In [8]:
rdd_paises3 = rdd_paises2.map(lambda x: (
    int(x[0]),     # id_pais
    x[1],          # nombre
    x[2]           # continente
))
rdd_paises3.take(5)

[(1, '30. Februar', 'AUT'),
 (2, 'A North American Team', 'MEX'),
 (3, 'Acipactli', 'MEX'),
 (4, 'Acturus', 'ARG'),
 (5, 'Afghanistan', 'AFG')]

In [9]:
schema_paises = StructType([
    StructField("id_pais", IntegerType(), False),
    StructField("nombre_pais", StringType(), False),
    StructField("continente", StringType(), False)
])


In [10]:
df_paises = spark.createDataFrame(rdd_paises3, schema=schema_paises)
df_paises


DataFrame[id_pais: int, nombre_pais: string, continente: string]

In [11]:
df_paises.printSchema()


root
 |-- id_pais: integer (nullable = false)
 |-- nombre_pais: string (nullable = false)
 |-- continente: string (nullable = false)



In [12]:
df_paises.show(5)

+-------+--------------------+----------+
|id_pais|         nombre_pais|continente|
+-------+--------------------+----------+
|      1|         30. Februar|       AUT|
|      2|A North American ...|       MEX|
|      3|           Acipactli|       MEX|
|      4|             Acturus|       ARG|
|      5|         Afghanistan|       AFG|
+-------+--------------------+----------+
only showing top 5 rows


In [13]:
# 1) Leer el archivo
rdd_juegos = spark.sparkContext.textFile("/home/jovyan/data/juegos.csv")
rdd_juegos.take(5)

[',nombre_juego,annio,temporada,ciudad',
 '1,1896 Verano,1896,Verano,Athina',
 '2,1900 Verano,1900,Verano,Paris',
 '3,1904 Verano,1904,Verano,St. Louis',
 '4,1906 Verano,1906,Verano,Athina']

In [14]:
# 2) Quitar encabezado
header_j = rdd_juegos.first()
rdd_juegos2 = rdd_juegos.filter(lambda x: x != header_j).map(lambda x: x.split(","))

In [15]:
# 3) Convertir tipos
rdd_juegos3 = rdd_juegos2.map(lambda x: (
    int(x[0]),   # id_juego
    x[1],        # nombre_juego
    x[2],        # anio
    x[3],        # temporada
    x[4]         # ciudad
))

In [16]:
# 4) Crear schema
schema_juegos = StructType([
    StructField("id_juego", IntegerType(), False),
    StructField("nombre_juego", StringType(), False),
    StructField("anio", StringType(), False),
    StructField("temporada", StringType(), False),
    StructField("ciudad", StringType(), False)
])

In [17]:
# 5) Crear DataFrame
df_juegos = spark.createDataFrame(rdd_juegos3, schema=schema_juegos)

In [18]:
# 6) Mostrar resultados
df_juegos.printSchema()
df_juegos.show(10)

root
 |-- id_juego: integer (nullable = false)
 |-- nombre_juego: string (nullable = false)
 |-- anio: string (nullable = false)
 |-- temporada: string (nullable = false)
 |-- ciudad: string (nullable = false)

+--------+-------------+----+---------+------------+
|id_juego| nombre_juego|anio|temporada|      ciudad|
+--------+-------------+----+---------+------------+
|       1|  1896 Verano|1896|   Verano|      Athina|
|       2|  1900 Verano|1900|   Verano|       Paris|
|       3|  1904 Verano|1904|   Verano|   St. Louis|
|       4|  1906 Verano|1906|   Verano|      Athina|
|       5|  1908 Verano|1908|   Verano|      London|
|       6|  1912 Verano|1912|   Verano|   Stockholm|
|       7|  1920 Verano|1920|   Verano|   Antwerpen|
|       8|1924 Invierno|1924| Invierno|    Chamonix|
|       9|  1924 Verano|1924|   Verano|       Paris|
|      10|1928 Invierno|1928| Invierno|Sankt Moritz|
+--------+-------------+----+---------+------------+
only showing top 10 rows


In [7]:
# 1. Leer archivo
rdd_resultados = spark.sparkContext.textFile("/home/jovyan/data/resultados.csv")
print("Archivo cargado:")
print(rdd_resultados.take(5))

Archivo cargado:
['resultado_id,medalla,deportista_id,juego_id,evento_id', '1,NA,1,39,1', '2,NA,2,49,2', '3,NA,3,7,3', '4,Gold,4,2,4']


In [8]:
# 2. Quitar encabezado
header = rdd_resultados.first()
rdd_resultados2 = rdd_resultados.filter(lambda x: x != header)

In [10]:
# 3. Separar por comas
rdd_resultados3 = rdd_resultados2.map(lambda x: x.split(","))
print("Después del split:")
print(rdd_resultados3.take(5))

Después del split:
[['1', 'NA', '1', '39', '1'], ['2', 'NA', '2', '49', '2'], ['3', 'NA', '3', '7', '3'], ['4', 'Gold', '4', '2', '4'], ['5', 'NA', '5', '36', '5']]


In [12]:
# 4. Convertir tipos
rdd_resultados_final = rdd_resultados3.map(lambda x: (
    int(x[0]),  # resultado_id
    x[1],       # medalla
    int(x[2]),  # deportista_id
    int(x[3]),  # juego_id
    int(x[4])   # evento_id
))

print("Casteado final:")
print(rdd_resultados_final.take(5))

Casteado final:
[(1, 'NA', 1, 39, 1), (2, 'NA', 2, 49, 2), (3, 'NA', 3, 7, 3), (4, 'Gold', 4, 2, 4), (5, 'NA', 5, 36, 5)]


In [13]:
# 5. Schema
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema_resultados_Torres = StructType([
    StructField("resultado_id", IntegerType(), False),
    StructField("medalla", StringType(), False),
    StructField("deportista_id", IntegerType(), False),
    StructField("juego_id", IntegerType(), False),
    StructField("evento_id", IntegerType(), False)
])

In [14]:
# 6. Crear DataFrame
df_resultados_Torres = spark.createDataFrame(rdd_resultados_final, schema=schema_resultados_Torres)

In [15]:
# 7. Ver salidas
df_resultados_Torres.printSchema()
df_resultados_Torres.show(10)

root
 |-- resultado_id: integer (nullable = false)
 |-- medalla: string (nullable = false)
 |-- deportista_id: integer (nullable = false)
 |-- juego_id: integer (nullable = false)
 |-- evento_id: integer (nullable = false)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
|           6|     NA|            5|      36|        6|
|           7|     NA|            5|      38|        5|
|           8|     NA|            5|      38|        6|
|           9|     NA|            5|      40|        5|
|          10|     NA|            5|      40|        6|
+------------+-------+-------------+--------+---

In [3]:
# === 1. Leer archivo ===
rdd_deporte = spark.sparkContext.textFile("/home/jovyan/data/deporte.csv")
print("Archivo deporte.csv cargado:")
print(rdd_deporte.take(5))

Archivo deporte.csv cargado:


[Stage 0:>                                                          (0 + 1) / 1]

['deporte_id,deporte', '1,Basketball', '2,Judo', '3,Football', '4,Tug-Of-War']


In [4]:
# === 2. Quitar encabezado ===
header_deporte = rdd_deporte.first()
rdd_deporte2 = rdd_deporte.filter(lambda x: x != header_deporte)

In [5]:
# === 3. Separar columnas ===
rdd_deporte3 = rdd_deporte2.map(lambda x: x.split(","))
print("Después del split:")
print(rdd_deporte3.take(5))

Después del split:
[['1', 'Basketball'], ['2', 'Judo'], ['3', 'Football'], ['4', 'Tug-Of-War'], ['5', 'Speed Skating']]


In [6]:
# === 4. Casteo de datos ===
# Estructura real del archivo:
# deporte_id, deporte
rdd_deporte_final = rdd_deporte3.map(lambda x: (
    int(x[0]),   # deporte_id
    x[1]         # nombre_deporte
))

print("Casteo final:")
print(rdd_deporte_final.take(5))


Casteo final:
[(1, 'Basketball'), (2, 'Judo'), (3, 'Football'), (4, 'Tug-Of-War'), (5, 'Speed Skating')]


In [7]:
# === 5. Crear Schema ===
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema_deporte_Torres = StructType([
    StructField("deporte_id", IntegerType(), False),
    StructField("nombre_deporte", StringType(), False)
])

In [8]:
# === 6. Crear DataFrame ===
df_deporte_Torres = spark.createDataFrame(rdd_deporte_final, schema=schema_deporte_Torres)

In [9]:
# === 7. Verificar ===
df_deporte_Torres.printSchema()
df_deporte_Torres.show(5)

root
 |-- deporte_id: integer (nullable = false)
 |-- nombre_deporte: string (nullable = false)

+----------+--------------+
|deporte_id|nombre_deporte|
+----------+--------------+
|         1|    Basketball|
|         2|          Judo|
|         3|      Football|
|         4|    Tug-Of-War|
|         5| Speed Skating|
+----------+--------------+
only showing top 5 rows


In [4]:
rdd_evento = spark.sparkContext.textFile("/home/jovyan/data/evento.csv")
rdd_evento.take(5)


['evento_id,evento,deporte_id',
 "1,Basketball Men's Basketball,1",
 "2,Judo Men's Extra-Lightweight,2",
 "3,Football Men's Football,3",
 "4,Tug-Of-War Men's Tug-Of-War,4"]

In [5]:
header = rdd_evento.first()
rdd_evento2 = rdd_evento.filter(lambda x: x != header)

In [6]:
rdd_evento3 = rdd_evento2.map(lambda x: x.replace('"', ''))


In [7]:
rdd_evento4 = rdd_evento3.map(lambda x: x.split(","))


In [8]:
def limpiar_evento(row):
    try:
        return (int(row[0]), row[1], int(row[2]))
    except:
        return None

rdd_evento5 = rdd_evento4.map(limpiar_evento).filter(lambda x: x is not None)


In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema_evento = StructType([
    StructField("evento_id", IntegerType(), False),
    StructField("nombre_evento", StringType(), False),
    StructField("deporte_id", IntegerType(), False)
])


In [10]:
df_evento_Torres = spark.createDataFrame(rdd_evento5, schema_evento)


In [11]:
df_evento_Torres.printSchema()
df_evento_Torres.show(10)


root
 |-- evento_id: integer (nullable = false)
 |-- nombre_evento: string (nullable = false)
 |-- deporte_id: integer (nullable = false)

+---------+--------------------+----------+
|evento_id|       nombre_evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
|        7|Cross Country Ski...|         6|
|        8|Cross Country Ski...|         6|
|        9|Cross Country Ski...|         6|
|       10|Cross Country Ski...|         6|
|       11|Cross Country Ski...|         6|
+---------+--------------------+----------+
only showing top 10 rows
